In [6]:
from openai import OpenAI
import requests
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv("local.env")


def review_code_with_gpt4(code_diff, openai_api_key):
    """
    Sends the code diff to GPT-4 and receives feedback.
    
    :param code_diff: The code differences as a string.
    :param openai_api_key: Your OpenAI API key.
    :return: The feedback from GPT-4 as a string.
    """
    client = OpenAI(api_key = openai_api_key)
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {
                    "role":"user", 
                    "content":f"Review the following github PR code changes:\n\n{code_diff}\n\nProvide detailed constructive feedback to improve the code given. When suggesting changes, please show the line before and after change."
                }
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Failed to get feedback from GPT-4."
    
# Function to fetch PR details including the diff
def fetch_pr_diff(owner, repo, pr_number, github_token=None):
    url = f"https://api.github.com/repos/{owner}/{repo}/pulls/{pr_number}"
    headers = {"Accept": "application/vnd.github.v3.diff"}
    if github_token:
        headers["Authorization"] = f"token {github_token}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.text  # Returns the diff of the PR
    else:
        print("Failed to fetch PR details. Status Code:", response.status_code)
        return None

def filter_diff_for_sql_and_py(diff_lines):
    """
    Filters out diffs from .ipynb files, focusing on .sql and .py files.

    :param diff_lines: A list of lines from the diff.
    :return: A list of filtered lines, excluding .ipynb file diffs.
    """
    filtered_diff = []
    skip_chunk = False

    for line in diff_lines:
        if line.startswith('diff --git'):
            # Check if this diff chunk is for an .ipynb file
            skip_chunk = '.ipynb' in line
        if not skip_chunk:
            filtered_diff.append(line)

    return filtered_diff




def split_diff_and_review(code_diff, openai_api_key, max_chars=24000):
    """
    Splits the diff into chunks based on a character count approximation to stay within token limits.
    """
    # Initialize variables
    feedbacks = []
    chunk = ""

    # Splitting the diff more accurately by character count
    for line in code_diff:
        # Check if adding the next line would exceed the max character count
        if len(chunk) + len(line) + 1 > max_chars:  # +1 for newline character
            print(chunk)
            # Review the current chunk
            feedback = review_code_with_gpt4(chunk, openai_api_key)
            feedbacks.append(feedback)
            # Start a new chunk
            chunk = line
        else:
            # Add the line to the current chunk
            chunk += f"\n{line}"

    # Don't forget to process the last chunk if it's not empty
    if chunk:
        feedback = review_code_with_gpt4(chunk, openai_api_key)
        feedbacks.append(feedback)

    return " ".join(feedbacks)

# Example usage
owner = "wkda"
repo = "ds-damage-vip-services"
pr_number = 137
github_token = os.getenv("GITHUB_API_KEY")  # Replace with your GitHub token if needed
openai_api_key = os.getenv("OPEN_AI_API_KEY")  # Replace with your OpenAI API key

print(f"Fetching PR #{pr_number} diff from {owner}/{repo}...")
pr_diff = fetch_pr_diff(owner, repo, pr_number, github_token)
print("Processing diff code")
pr_diff_filtered = filter_diff_for_sql_and_py(pr_diff.split("\n"))
print("Reviewing code diff")
feedback = split_diff_and_review(pr_diff_filtered, openai_api_key)



Fetching PR #137 diff from wkda/ds-damage-vip-services...
Processing diff code
Reviewing code diff


In [7]:
print(feedback)

The code changes made in this PR are quite well done and sensible. However, there are a few small recommendations to further improve the clarity and readability of the code.

1. In `log_setup.py,` the `create_tracing(request.headers)` methods return b3 headers which are then used to extract trace_id and span_id. The function name `create_tracing` implies it's creating something rather than fetching the tracing headers, so this could potentially be renamed to better represent its action.

```python
- b3_headers = create_tracing(request.headers)
+ b3_headers = get_tracing_headers(request.headers)
```

2. When catching the response exception in `log_setup.py`, a logger.error is used. However, this does not provide sufficient information about the error. The captured exception is not added to the log message. This can be improved by adding exception to the log message using `exc_info=True`.

```python
- self.logger.error("Response exception occurred")
+ self.logger.error("Response exceptio